# Langchain: The basics

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [1]:
!pip -q install openai langchain langchain_openai langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00


In [2]:
import os
from getpass import getpass

In [3]:
#os.environ['OPENAI_API_KEY'] = getpass('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


## Zero Shot Prompting



In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [ ]:
#llm = ChatOpenAI(model_name='gpt-3.5-turbo',
#             temperature=0.2,
#             max_tokens = 256)

In [5]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=256,
    max_retries=2,
)

In [6]:
text = """What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:"""

print(llm.invoke(text))

content='Sentiment: Positive. The customer uses words like "fresh", "flavorful", and "loved" to describe their experience, indicating a positive sentiment towards the restaurant.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 80, 'total_tokens': 117, 'completion_time': 0.134545455, 'prompt_time': 0.006053522, 'queue_time': 0.225291843, 'total_time': 0.140598977}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_e669a124b2', 'finish_reason': 'stop', 'logprobs': None} id='run-febfa9f5-3bf3-4d3e-a5b0-e6a4fafab335-0' usage_metadata={'input_tokens': 80, 'output_tokens': 37, 'total_tokens': 117}


## Prompting with Prompt Templates

In [7]:
from langchain import PromptTemplate

In [8]:
restaurant_template = """
What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: {review_text}
Sentiment:
"""

review_prompt = PromptTemplate(
    input_variables=["review_text"],
    template=restaurant_template,
)

In [9]:
reviews = [
    "The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.",
    "Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.",
    "The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!",
    "The food tasted alright, but the tables were not very clean, which was off-putting.",
    "Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit."
]

In [10]:
print(review_prompt.format(review_text=reviews[0]))


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:



In [11]:
## querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=review_prompt)

<ipython-input-11-7b8f9a9dc912>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=review_prompt)


In [12]:
chain.invoke({"review_text": reviews[0]})

{'review_text': 'The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.',
 'text': 'Sentiment: Positive. The customer uses words like "fresh", "flavorful", "loved", and mentions the "chic décor" and "ambiance", indicating a very favorable opinion of their experience.'}

In [13]:
for review in reviews:
    print(review_prompt.format(review_text=review))
    print(chain.invoke({"review_text": review}))
    print("================================")


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:

{'review_text': 'The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.', 'text': 'Sentiment: Positive. The customer uses words like "fresh", "flavorful", "loved", and mentions the "chic décor" and "ambiance", indicating a very favorable opinion of their experience.'}

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Sentiment:

{'review_text': 'Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.', 'text': 'Sentiment: Positive'}

What is the sentiment of the customer review given below? It sh

## Prompting ith Few Shot Learning

In [14]:
from langchain import FewShotPromptTemplate

In [15]:
# First, create the list of few shot examples.
examples = [
    {
        "Review": "The grilled chicken was seasoned to perfection and simply melted in the mouth.",
        "Category": "Food Quality"
    },
    {
        "Review": "Despite the crowd, the place was immaculately clean and the restrooms were spotless.",
        "Category": "Overall Hygiene"
    },
    {
        "Review": "The dim lighting and soothing jazz music provided an intimate and romantic setting.",
        "Category": "Restaurant Ambience"
    },
    {
        "Review": "We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.",
        "Category": "Customer Service"
    }
]

In [16]:
# Prepare the prompt template
example_formatter_template = """
review: {Review}
category: {Category}\n
"""
category_prompt = PromptTemplate(
    input_variables=["Review", "Category"],
    template=example_formatter_template,
)

In [17]:
print(category_prompt.format(**examples[0]))


review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality




In [18]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_category_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=category_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Classify the reviews into one the four categories as given in the examples.",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Review: {review_text}\nCategory:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["review_text"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [19]:
# We can now generate a prompt using the `format` method.
print(few_shot_category_prompt.format(review_text=reviews[0]))

Classify the reviews into one the four categories as given in the examples.

review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality



review: Despite the crowd, the place was immaculately clean and the restrooms were spotless.
category: Overall Hygiene



review: The dim lighting and soothing jazz music provided an intimate and romantic setting.
category: Restaurant Ambience



review: We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.
category: Customer Service


Review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Category:


In [20]:
from langchain.chains import LLMChain

category_chain = LLMChain(llm=llm, prompt=few_shot_category_prompt)

print(reviews[0])
# Run the chain only specifying the input variable.
print(category_chain.run(reviews[0]))

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.


<ipython-input-20-6e42368bd0d4>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(category_chain.run(reviews[0]))


Based on the given examples, this review can be classified into two categories, but since it has to be classified into one, I would choose: 

Category: Food Quality 

Reason: Although the review mentions the ambiance, the primary focus is on the food ("The seafood platter was fresh and flavorful").


In [21]:
for review in reviews:
    print(review)
    print(category_chain.run({"review_text": review}))
    print("================================")

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Based on the given examples, this review can be classified into two categories, but since it has to be classified into one, I would choose: 

Category: Food Quality 

Reason: Although the review mentions the ambiance, the primary focus is on the food ("The seafood platter was fresh and flavorful").
Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Category: Customer Service
The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Based on the given examples, I would categorize the review as:

Category: Restaurant Ambience

The review mentions the restaurant's interior and the visual appeal, which aligns with the "Restaurant Ambience" category. Although it also mentions the gourmet dishes, the primary focus is on the ambiance and the overall experience, rather than just t

## Chaining Multiple Prompts

In [22]:
from langchain.schema import StrOutputParser

In [23]:
review_chain = review_prompt | llm | StrOutputParser()

In [24]:
review_chain.invoke({"review_text": reviews[0]})

'Sentiment: Positive. The customer uses words like "fresh", "flavorful", "loved", and mentions the "chic décor" and "ambiance", indicating a very favorable opinion of their experience.'

In [25]:
category_chain = few_shot_category_prompt | llm | StrOutputParser()

In [26]:
category_chain.invoke({"review_text": reviews[1]})

'Category: Customer Service'

In [27]:
response_to_customer = """
"Write an appropriate response to the customer by either appreciating their positive experience or adressing the specific concern that customer might
have mentioned in the feedback below.

<feedback>
{feedback}
</feedback>

The feedback is about {category}.

The goal of the response is to ensure customer engagement.
"""

In [28]:
response_prompt = PromptTemplate(
    input_variables=["feedback", "category"],
    template=response_to_customer,
)

In [29]:
final_response_chain = response_prompt | llm | StrOutputParser()

In [30]:
complete_chain = (
    {
        "feedback": review_chain,
        "category": category_chain,
    }
    | final_response_chain
)

In [31]:
from pprint import pprint

In [32]:
reviews[3]

'The food tasted alright, but the tables were not very clean, which was off-putting.'

In [33]:
pprint(complete_chain.invoke({"review_text": reviews[4]}))

('Dear valued customer,\n'
 '\n'
 'I am sorry to hear that your recent experience at our establishment did not '
 'meet your expectations. Specifically, I apologize for the wait you '
 'encountered despite having a reservation, and for the main course being '
 'served cold. I understand how frustrating it can be to wait for a table, '
 'especially when you have taken the time to make a reservation.\n'
 '\n'
 'I want to assure you that we take these issues seriously and are taking '
 'immediate action to address them. We are reviewing our reservation and '
 'seating processes to ensure that our customers are seated promptly, and we '
 'are re-training our staff to ensure that our food is served at the correct '
 'temperature.\n'
 '\n'
 'I would like to invite you to give us another chance to show you the level '
 'of service and quality that we strive to provide. If you could please '
 'contact me directly, I would be happy to offer you a complimentary meal on '
 'your next visit.\n'
 '